In [1]:
import os,io
import pandas as pd
import cv2
from google.cloud import vision
from google.cloud.vision import types
from pandas.io.json import json_normalize

os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r'ServiceAccountToken.json'
client = vision.ImageAnnotatorClient()

file_name = 'test.jpg'
image_path ='Images/test.jpg'

#To get Image Shape
img=cv2.imread(image_path)
fh,fw,c=img.shape

#Open file from path
with io.open(image_path,'rb') as image_file:
    content = image_file.read()

# construct an image instance
image = vision.types.Image(content=content)

# annotate Image Response : this would be in JSON format
response = client.text_detection(image=image)  # returns TextAnnotation
df = pd.DataFrame(columns=['locale', 'description'])

texts = response.text_annotations

for text in texts:
    df = df.append(
        dict(
            locale=text.locale,
            description=text.description
        ),
        ignore_index=True
    )
    v=text.bounding_poly.vertices

def Bounding_Box(response):
    df1 = pd.DataFrame(columns=['Text', 'xp', 'yp','x2p','y2p','xcp','ycp','wp','hp'])

    i=0
    for text in response.text_annotations:
        if "\n" in text.description:
            continue

        j=0
        for v in text.bounding_poly.vertices:
            if j==1:
                right_bottom_x=v.x
                right_bottom_y=v.y
            if j==3:
                top_left_x=v.x
                top_left_y=v.y
            j+=1

        #Top-left and Bottom-right Coordinates
        xl=top_left_x
        yl=top_left_y
        xr=right_bottom_x
        yr=right_bottom_y

        #Center Coordinates
        xc=(xl+yr)/2.0
        yc=(yl+yr)/2.0

        #Dimension of Bounding-Box
        w=abs(xr-xl)
        h=abs(yr-yl)

        #Normalize the coordinates
        xl/=fw
        yl/=fh
        xr/=fw
        yr/=fh
        xc/=fw
        yc/=fh
        w/=fw
        h/=fh

        #Push in Dataframe
        df1.loc[i] = [text.description] + [xl,yl,xr,yr,xc,yc,w,h]
        i+=1
        
    
    return df1
    
#Coordinates will be a Dataframe
Coordinates=Bounding_Box(response)

print(Coordinates)

        Text        xp        yp       x2p       y2p       xcp       ycp  \
0        100  0.114103  0.093750  0.178205  0.023438  0.062821  0.058594   
1         95  0.135897  0.286458  0.178205  0.221354  0.122436  0.253906   
2         90  0.135897  0.486979  0.178205  0.421875  0.171795  0.454427   
3         85  0.135897  0.682292  0.178205  0.617188  0.219872  0.649740   
4  2.65-3.99  0.251282  0.812500  0.426923  0.747396  0.309615  0.779948   
5  4.00-5.99  0.514103  0.812500  0.689744  0.750000  0.441667  0.781250   
6   6.00-Top  0.773077  0.825521  0.946154  0.750000  0.571154  0.787760   
7    C-value  0.523077  0.981771  0.676923  0.919271  0.487821  0.950521   
8  precision  0.048718  0.539062  0.007692  0.171875  0.066667  0.355469   

         wp        hp  
0  0.064103  0.070312  
1  0.042308  0.065104  
2  0.042308  0.065104  
3  0.042308  0.065104  
4  0.175641  0.065104  
5  0.175641  0.062500  
6  0.173077  0.075521  
7  0.153846  0.062500  
8  0.041026  0.367188  